In [1]:
import numpy as np
from warcio.archiveiterator import ArchiveIterator
import re
import requests
import sys
from pprint import pprint
import codecs
import urllib.request
import gzip
import glob
import threading
import time

In [2]:
# beginning months: '05', '10', '16', 
for x in ['24', '29', '34', '40']:
    print("http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-{}/warc.paths.gz".format(x))
    urllib.request.urlretrieve("http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-{}/warc.paths.gz".format(x), 'paths/{}'.format(x))

http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-24/warc.paths.gz
http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-29/warc.paths.gz
http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-34/warc.paths.gz
http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/warc.paths.gz


In [3]:
crawl_paths = []
for segment_files in glob.glob('paths/*'):
    file = gzip.open(segment_files)
    urls = ["http://commoncrawl.s3.amazonaws.com/" + x for x in str(file.read(), 'utf-8').split('\n')]
    crawl_paths.extend(urls)
crawl_paths[:5]

['http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400187354.1/warc/CC-MAIN-20200918061627-20200918091627-00000.warc.gz',
 'http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400187354.1/warc/CC-MAIN-20200918061627-20200918091627-00001.warc.gz',
 'http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400187354.1/warc/CC-MAIN-20200918061627-20200918091627-00002.warc.gz',
 'http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400187354.1/warc/CC-MAIN-20200918061627-20200918091627-00003.warc.gz',
 'http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-40/segments/1600400187354.1/warc/CC-MAIN-20200918061627-20200918091627-00004.warc.gz']

In [41]:
def get_hits(file_name):
#     regex = re.compile("(\bcovid|\becon|\bcoronavirus)", re.IGNORECASE)
#     regex = re.compile("(\bcovid|\bcorona)", re.IGNORECASE)
#     regex = re.compile(
#         "(youtu\.be/|youtube\.com/(watch\?(.*\&)?v=|(embed|v)/))([^?&\"'>]+)"
#     )
    regex = re.compile('covid', re.IGNORECASE)

    entries, matching_entries, hits = 0, 0, 0
    stream = requests.get(file_name, stream=True).raw
    
    records = []
    for record in ArchiveIterator(stream):
        if record.rec_type == "warcinfo":
            continue

        if not ".com/" in record.rec_headers.get_header("WARC-Target-URI"):
            continue
        entries = entries + 1
        contents = record.content_stream().read(1000).decode("utf-8", "replace")
        m = regex.search(contents)
        if m:
            records.append(record)
            print(records)
    return records

In [36]:
def gather_records(threadName, threadNum, crawl_paths):
    print((threadNum, len(crawl_paths), num_threads))
    num_warcs_to_read = 2
    for i in range(num_warcs_to_read):
        path = crawl_paths[i*num_threads + threadNum]
        print("{} iter {}: {}\n".format(threadName, i, len(records) ))
        thread_results[threadNum].extend(records)

In [37]:
num_threads = 8
threads = []
thread_results = [[] for _ in range(num_threads)]
for i in range(num_threads):
    x = threading.Thread(target=gather_records, args=("Thread-{}".format(i), i, crawl_paths))
    threads.append(x)
    x.start()

(0, 259604, 8)
(1, 259604, 8)
(2, 259604, 8)
(3, 259604, 8)(4, 259604, 8)
(5, 259604, 8)
(6, 259604, 8)

(7, 259604, 8)


In [ ]:
urls = []
for lst in thread_results:
    for record in lst:
        if check_criteria(record):
            urls.append(get_url(record))